In [ ]:
import pandas as pd
import json

# Load vulnerability dataset
vuln_df = pd.read_csv("vuln_gpt.csv")

def json_load(x):
    try:
        return json.loads(x)
    except:
        return None

# Ensure vuln_loc column is properly parsed
vuln_df["vuln_loc"] = vuln_df["vuln_loc"].apply(json_load)
vuln_df.dropna(inplace=True)

In [ ]:
import re

data = [
    'concentratedliquidityposition.sol#burn',
    'concentratedliquiditypool.sol#_updateposition',
    'concentratedliquiditypool.sol#burn',
    'ticks.cross',
    'concentratedliquidityposition#collect'
]

# Regex to match the part after the last '#' or '.'
pattern = r'[^#.]+$'

results = [re.search(pattern, item).group() for item in data]

print(results)

['burn', '_updateposition', 'burn', 'cross', 'collect']


In [3]:
import re

pattern = r'[^#.]+$'

# Update each row in-place
for idx, row in vuln_df.iterrows():
    data = row['vuln_loc']['vuln_function']
    if isinstance(data, list):  # In case vuln_function is a list
        results = [re.search(pattern, item).group() for item in data if re.search(pattern, item)]
        results = list(set(results))
        vuln_df.at[idx, 'vuln_loc']['vuln_function'] = results

In [4]:
vuln_df

,file_name,vuln_title,severity,description,vuln_loc,vuln_explanation,vuln_recommendation
0,35.md,Unsafe cast in `ConcentratedLiquidityPool.burn...,high,The `ConcentratedLiquidityPool.burn` function ...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,The `ConcentratedLiquidityPool.burn` function ...,"Even though Solidity 0.8.x is used, type casts..."
1,35.md,Wrong usage of `positionId` in `ConcentratedLi...,high,#### Impact\nIn the `subscribe` function of `C...,{'vuln_file': ['ConcentratedLiquidityPoolManag...,In the `subscribe` function of `ConcentratedLi...,Change `positionId` to `incentiveId` in the re...
2,35.md,`ConcentratedLiquidityPoolManager`'s incentive...,high,The `ConcentratedLiquidityPoolManager` keeps a...,{'vuln_file': ['ConcentratedLiquidityPoolManag...,The `ConcentratedLiquidityPoolManager` keeps a...,"In `reclaimIncentive`, reduce `incentive.rewar..."
3,35.md,Overflow in the `mint` function of `Concentrat...,high,#### Impact\nSimilar to a previous finding in ...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,Similar to a previous finding in the `IndexPoo...,Consider removing the `unchecked` statement to...
4,35.md,Incorrect usage of typecasting in `_getAmounts...,high,#### Impact\nThe `_getAmountsForLiquidity` fun...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,The `_getAmountsForLiquidity` function of `Con...,Check whether the result of `DyDxMath.getDy` o...
...,...,...,...,...,...,...,...
1739,52.md,Unclear `TwapOracle.consult` algorithm,medium,The `TWAPOracle.consult` function is unclear t...,"{'vuln_file': ['TWAPOracle.sol'], 'vuln_functi...",The `TWAPOracle.consult` function is unclear t...,Document how the algorithm works and make sure...
1740,52.md,Tokens with fee on transfer are not supported,medium,There are ERC20 tokens that charge fee for eve...,"{'vuln_file': ['BasePoolV2.sol'], 'vuln_functi...",There are ERC20 tokens that charge fee for eve...,Consider calling `balanceOf()` to get the actu...
1741,52.md,VaderPoolV2.rescue results in loss of funds ra...,medium,#### Impact\n\nAny unaccounted for tokens on `...,"{'vuln_file': ['BasePoolV2.sol'], 'vuln_functi...",Any unaccounted for tokens on `VaderPoolV2` ca...,Permission this function to only allow Vader g...
1742,52.md,No way to remove GasThrottle after deployment,medium,#### Impact\n\nPotential DOS on swaps\n\n#### ...,"{'vuln_file': ['GasThrottle.sol'], 'vuln_funct...",BasePool and BasePoolV2 make use of a `validat...,Either remove GasThrottle.sol entirely or allo...


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from solidity_parser import parser

# Base directory containing smart contract projects
BASE_DIR = "Web3Bugs/contracts/"

# Prepare CSV files for storing results
vuln_code_data = []
non_vuln_code_data = []

columns = list(vuln_df.columns) + ["vuln_file_path", "vuln_code"]

def safe_serialize(obj):
    """Convert Solidity parser output to a JSON-serializable format."""
    if isinstance(obj, dict):
        return {k: safe_serialize(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [safe_serialize(v) for v in obj]
    elif hasattr(obj, '__dict__'):
        return safe_serialize(obj.__dict__)
    return obj

def extract_function_names(parsed_code):
    functions = []
    def traverse_nodes(nodes):
        for node in nodes:
            if isinstance(node, dict):
                if node.get('type') == 'FunctionDefinition':
                    functions.append(node.get('name', 'Unnamed').strip().lower())
                traverse_nodes(node.values())
            elif isinstance(node, list):
                traverse_nodes(node)
    traverse_nodes(parsed_code.get('children', []))
    return functions

def normalize_function_name(name):
    return name.strip().lower().replace(" ", "").replace("\t", "").replace("\n", "")

def clean_function_list(functions):
    """Deduplicate and clean function names from a list."""
    cleaned_functions = set()
    for func in functions:
        func_name = func.split("(")[0].strip().lower()
        cleaned_functions.add(func_name)
    return list(cleaned_functions)

def extract_vuln_code(file_path, functions):
    """Extract vulnerable function or event-related code using Solidity parser."""
    if not os.path.isfile(file_path):
        return None

    try:
        parsed_code = parser.parse_file(file_path, loc=True)
    except Exception as e:
        print(f"Parsing error in {file_path}: {e}")
        return None

    extracted_code = []

    # Read the entire file for accurate line extraction
    with open(file_path, "r", encoding="utf-8") as f:
        code_lines = f.readlines()

    available_functions = extract_function_names(parsed_code)
    functions = clean_function_list(functions)
    available_functions = clean_function_list(available_functions)

    def traverse_nodes(nodes):
        for node in nodes:
            if isinstance(node, dict):
                if node.get('type') == 'FunctionDefinition':
                    function_name = normalize_function_name(node.get('name', ''))
                    if function_name in functions:
                        start = node['loc']['start']['line'] - 1  # Convert to zero-based index
                        end = node['loc']['end']['line']          # Solidity Parser returns 1-based index
                        function_code = "".join(code_lines[start:end]).strip()
                        if not function_code.endswith("}"):
                            function_code += "  // Error: Function body might be cropped."
                        extracted_code.append(function_code)
                traverse_nodes(node.values())
            elif isinstance(node, list):
                traverse_nodes(node)

    traverse_nodes(parsed_code.get('children', []))

    if not extracted_code:
        print(f"\nNo match found for {functions} in {file_path}. Available functions: {available_functions}\n")

    return "\n\n".join(extracted_code) if extracted_code else None

def list_solidity_files(directory):
    """Recursively list all Solidity files in a given directory."""
    sol_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".sol"):
                sol_files.append(os.path.join(root, file))
    return set(sol_files)

def find_vuln_file(project_path, vuln_file):
    """Recursively search for the vulnerability file within the project directory."""
    for root, _, files in os.walk(project_path):
        if vuln_file in files:
            return os.path.join(root, vuln_file)
    return None

count = 0

# Process each project
for _, row in tqdm(vuln_df.iterrows(), total=len(vuln_df), desc="Processing vulnerabilities"):
    project_id = row["file_name"].replace(".md", "")
    vuln_info = row["vuln_loc"]

    if not isinstance(vuln_info, dict):
        continue

    vuln_files = set(vuln_info.get("vuln_file", []))
    vuln_functions = vuln_info.get("vuln_function", [])

    project_path = os.path.join(BASE_DIR, project_id)
    if not os.path.isdir(project_path):
        continue

    # Recursively list all Solidity files in the project
    all_files = list_solidity_files(project_path)

    all_vuln_code = []

    # Extract vulnerable code
    for vuln_file in vuln_files:
        vuln_file_path = find_vuln_file(project_path, vuln_file)
        if vuln_file_path:
            vuln_code = extract_vuln_code(vuln_file_path, vuln_functions)
            all_vuln_code.append(vuln_code)

    clean_vuln_code = [x for x in all_vuln_code if x != None]

    if clean_vuln_code:
        final_vuln_code = "".join([x for x in clean_vuln_code]).strip()
        vuln_code_data.append(list(row) + [vuln_file_path, final_vuln_code])

# Step 1: Aggregating all vulnerable files per project ID
project_vuln_files = {}
for _, row in vuln_df.iterrows():
    project_id = row["file_name"].replace(".md", "")
    vuln_info = row["vuln_loc"]

    if isinstance(vuln_info, dict):
        vuln_files = set(vuln_info.get("vuln_file", []))
        if project_id not in project_vuln_files:
            project_vuln_files[project_id] = set()
        project_vuln_files[project_id].update(vuln_files)

# Step 2: Processing each project and calculating non-vulnerable files
for project_id, vuln_files in tqdm(project_vuln_files.items(), desc="Processing projects"):
    project_path = os.path.join(BASE_DIR, project_id)
    if not os.path.isdir(project_path):
        continue

    # List all Solidity files in the project
    all_files = list_solidity_files(project_path)
    non_vuln_files = all_files - {os.path.join(project_path, vf) for vf in vuln_files}

    # Extract non-vulnerable files
    for non_vuln_file in non_vuln_files:
        if os.path.isfile(non_vuln_file):
            with open(non_vuln_file, "r", encoding="utf-8") as f:
                non_vuln_code_data.append([project_id, non_vuln_file, f.read()])

# Save to CSV
vuln_code_df = pd.DataFrame(vuln_code_data, columns=columns)
non_vuln_code_df = pd.DataFrame(non_vuln_code_data, columns=["project_id", "file_path", "code"])

print("Extraction completed.")

Processing vulnerabilities:   0%|          | 8/1703 [00:11<45:28,  1.61s/it]  


No match found for ['rangefeegrowth'] in Web3Bugs/contracts/35/trident/contracts/libraries/concentratedPool/Ticks.sol. Available functions: ['remove', 'cross', 'getmaxliquidity', 'insert']



Processing vulnerabilities:   1%|          | 10/1703 [00:16<51:42,  1.83s/it]


No match found for [] in Web3Bugs/contracts/35/trident/contracts/pool/concentrated/ConcentratedLiquidityPool.sol. Available functions: ['swap', '_updatereserves', 'collect', '_updatefees', 'flashswap', 'gettokenprotocolfees', 'updatebarfee', '_balance', 'rangefeegrowth', 'collectprotocolfee', '_updateposition', 'getliquidityandlastobservation', '_ensuretickspacing', '_getamountsforliquidity', 'constructor', 'getamountout', 'getassets', 'rangesecondsinside', 'getpriceandnearestticks', 'mint', '_transferbothtokens', 'burn', 'getreserves', '_transfer', 'burnsingle', 'getimmutables']



Processing vulnerabilities:   1%|          | 16/1703 [00:19<21:24,  1.31it/s]

...

Parsing error in Web3Bugs/contracts/52/contracts/governance/GovernorAlpha.sol: unrecognized expression


Processing vulnerabilities: 100%|█████████▉| 1701/1703 [21:09<00:00,  2.09it/s]


No match found for ['validategas'] in Web3Bugs/contracts/52/contracts/dex/utils/GasThrottle.sol. Available functions: []



Processing projects: 100%|██████████| 101/101 [00:00<00:00, 384.75it/s]

Extraction completed.


In [6]:
vuln_code_df

,file_name,vuln_title,severity,description,vuln_loc,vuln_explanation,vuln_recommendation,vuln_file_path,vuln_code
0,35.md,Unsafe cast in `ConcentratedLiquidityPool.burn...,high,The `ConcentratedLiquidityPool.burn` function ...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,The `ConcentratedLiquidityPool.burn` function ...,"Even though Solidity 0.8.x is used, type casts...",Web3Bugs/contracts/35/trident/contracts/pool/c...,function burn(bytes calldata data) public over...
1,35.md,Wrong usage of `positionId` in `ConcentratedLi...,high,#### Impact\nIn the `subscribe` function of `C...,{'vuln_file': ['ConcentratedLiquidityPoolManag...,In the `subscribe` function of `ConcentratedLi...,Change `positionId` to `incentiveId` in the re...,Web3Bugs/contracts/35/trident/contracts/pool/c...,"function subscribe(uint256 positionId, uint256..."
2,35.md,`ConcentratedLiquidityPoolManager`'s incentive...,high,The `ConcentratedLiquidityPoolManager` keeps a...,{'vuln_file': ['ConcentratedLiquidityPoolManag...,The `ConcentratedLiquidityPoolManager` keeps a...,"In `reclaimIncentive`, reduce `incentive.rewar...",Web3Bugs/contracts/35/trident/contracts/pool/c...,function reclaimIncentive(\n IConcentra...
3,35.md,Overflow in the `mint` function of `Concentrat...,high,#### Impact\nSimilar to a previous finding in ...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,Similar to a previous finding in the `IndexPoo...,Consider removing the `unchecked` statement to...,Web3Bugs/contracts/35/trident/contracts/pool/c...,function mint(bytes calldata data) public over...
4,35.md,Incorrect usage of typecasting in `_getAmounts...,high,#### Impact\nThe `_getAmountsForLiquidity` fun...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,The `_getAmountsForLiquidity` function of `Con...,Check whether the result of `DyDxMath.getDy` o...,Web3Bugs/contracts/35/trident/contracts/librar...,function _getAmountsForLiquidity(\n uin...
...,...,...,...,...,...,...,...,...,...
942,52.md,Lacking Validation Of Chainlink' Oracle Queries,medium,#### Impact\n\n`TwapOracle.consult()` is missi...,"{'vuln_file': ['TwapOracle.sol', 'GasThrottle....",TwapOracle.consult() is missing additional val...,Consider validating the output of latestRoundD...,Web3Bugs/contracts/52/contracts/twap/TwapOracl...,function consult(address token) public view re...
943,52.md,Vests can be denied,medium,The `LinearVesting.vestFor` function (which is...,"{'vuln_file': ['LinearVesting.sol'], 'vuln_fun...",The `LinearVesting.vestFor` function (which is...,There are several ways to mitigate this.\nThe ...,Web3Bugs/contracts/52/contracts/tokens/vesting...,"function vestFor(address user, uint256 amount)..."
944,52.md,Tokens with fee on transfer are not supported,medium,There are ERC20 tokens that charge fee for eve...,"{'vuln_file': ['BasePoolV2.sol'], 'vuln_functi...",There are ERC20 tokens that charge fee for eve...,Consider calling `balanceOf()` to get the actu...,Web3Bugs/contracts/52/contracts/dex-v2/pool/Ba...,"function mint(\n IERC20 foreignAsset,\n..."
945,52.md,VaderPoolV2.rescue results in loss of funds ra...,medium,#### Impact\n\nAny unaccounted for tokens on `...,"{'vuln_file': ['BasePoolV2.sol'], 'vuln_functi...",Any unaccounted for tokens on `VaderPoolV2` ca...,Permission this function to only allow Vader g...,Web3Bugs/contracts/52/contracts/dex-v2/pool/Ba...,function rescue(IERC20 foreignAsset) external ...


In [7]:
non_vuln_code_df

,project_id,file_path,code
0,35,Web3Bugs/contracts/35/trident/contracts/librar...,// SPDX-License-Identifier: GPL-3.0-or-later\n...
1,35,Web3Bugs/contracts/35/trident/contracts/librar...,// SPDX-License-Identifier: GPL-3.0-or-later\n...
2,35,Web3Bugs/contracts/35/trident/contracts/pool/c...,// SPDX-License-Identifier: GPL-3.0-or-later\n...
3,35,Web3Bugs/contracts/35/trident/contracts/pool/f...,// SPDX-License-Identifier: GPL-3.0-or-later\n...
4,35,Web3Bugs/contracts/35/trident/contracts/librar...,// SPDX-License-Identifier: GPL-2.0-or-later\n...
...,...,...,...
6283,52,Web3Bugs/contracts/52/contracts/external/inter...,// SPDX-License-Identifier: Unlicense\npragma ...
6284,52,Web3Bugs/contracts/52/contracts/tokens/vesting...,// SPDX-License-Identifier: Unlicense\n\npragm...
6285,52,Web3Bugs/contracts/52/contracts/interfaces/dex...,// SPDX-License-Identifier: Unlicense\n\npragm...
6286,52,Web3Bugs/contracts/52/contracts/external/libra...,// SPDX-License-Identifier: GPL-3.0-or-later\n...


In [ ]:
vuln_code_df.to_csv("vulnerable_code.csv", index=False)
non_vuln_code_df.to_csv("non_vulnerable_code.csv", index=False)

In [10]:
# Removing test case
clean_non_vuln_df = non_vuln_code_df[non_vuln_code_df['file_path'].apply(lambda x: "test" not in str(x) if pd.notnull(x) else False)]

In [12]:
vuln_project_id = [int(x[:-3]) for x in vuln_code_df['file_name'].value_counts().index]
vuln_project_count = [int(x) for x in vuln_code_df['file_name'].value_counts().values]

clean_non_vuln_df['project_id'] = clean_non_vuln_df['project_id'].astype(int)

non_vuln_code_list = []

for idx, item in enumerate(vuln_project_id):
    sample_size = min(len(clean_non_vuln_df[clean_non_vuln_df['project_id'] == item]), vuln_project_count[idx]+3)
    sample_code = clean_non_vuln_df[clean_non_vuln_df['project_id'] == item].sample(sample_size, random_state=42)
    non_vuln_code_list.append(sample_code)

concatenated_df = pd.concat(non_vuln_code_list, ignore_index=True)
concatenated_df

/tmp/ipykernel_8346/920649238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_non_vuln_df['project_id'] = clean_non_vuln_df['project_id'].astype(int)


,project_id,file_path,code
0,52,Web3Bugs/contracts/52/contracts/interfaces/tok...,// SPDX-License-Identifier: Unlicense\n\npragm...
1,52,Web3Bugs/contracts/52/contracts/x-vader/XVader...,// SPDX-License-Identifier: MIT\npragma solidi...
2,52,Web3Bugs/contracts/52/contracts/interfaces/dex...,// SPDX-License-Identifier: Unlicense\n\npragm...
3,52,Web3Bugs/contracts/52/contracts/external/inter...,// SPDX-License-Identifier: Unlicense\npragma ...
4,52,Web3Bugs/contracts/52/contracts/external/libra...,// SPDX-License-Identifier: Unlicense\n\npragm...
...,...,...,...
1129,77,Web3Bugs/contracts/77/elasticswap/src/contract...,//SPDX-License-Identifier: GPL-3.0\npragma sol...
1130,53,Web3Bugs/contracts/53/contracts/MixinOperatorR...,// SPDX-License-Identifier: GPL-3.0-or-later\n...
1131,53,Web3Bugs/contracts/53/contracts/mocks/MockSmar...,
1132,53,Web3Bugs/contracts/53/contracts/interfaces/INe...,// SPDX-License-Identifier: GPL-3.0-or-later\n...


In [13]:
import random
from solidity_parser import parser
from tqdm import tqdm

def extract_function_names(parsed_code):
    functions = []
    def traverse_nodes(nodes):
        for node in nodes:
            if isinstance(node, dict):
                if node.get('type') == 'FunctionDefinition':
                    functions.append(node.get('name', 'Unnamed').strip().lower())
                traverse_nodes(node.values())
            elif isinstance(node, list):
                traverse_nodes(node)
    traverse_nodes(parsed_code.get('children', []))
    return functions

def normalize_function_name(name):
    return name.strip().lower().replace(" ", "").replace("\t", "").replace("\n", "")

def extract_non_vuln_code(code):
    """Extract vulnerable function or event-related code using Solidity parser."""
    try:
        parsed_code = parser.parse(code, loc=True)

        extracted_code = []

        available_functions = extract_function_names(parsed_code)
        sample_size = 1 if len(available_functions) == 1 else random.randint(1, min(len(available_functions),3))
        functions = random.sample(available_functions, sample_size)

        code_lines = code.splitlines(keepends=True)

        def traverse_nodes(nodes):
            for node in nodes:
                if isinstance(node, dict):
                    if node.get('type') == 'FunctionDefinition':
                        function_name = normalize_function_name(node.get('name', ''))
                        if function_name in functions:
                            start = node['loc']['start']['line'] - 1  # Convert to zero-based index
                            end = node['loc']['end']['line']          # Solidity Parser returns 1-based index
                            function_code = "".join(code_lines[start:end]).strip()
                            if not function_code.endswith("}"):
                                function_code += "  // Error: Function body might be cropped."
                            extracted_code.append(function_code)
                    traverse_nodes(node.values())
                elif isinstance(node, list):
                    traverse_nodes(node)

        traverse_nodes(parsed_code.get('children', []))

        return "\n\n".join(extracted_code) if extracted_code else None
    except Exception as e:
        print(f"Parsing error message {e}")
        return None

# Apply the function with a manual progress bar
concatenated_df["clean_code"] = [
    extract_non_vuln_code(row) for row in tqdm(concatenated_df["code"], desc="Processing")
]

Processing:   0%|          | 0/1134 [00:00<?, ?it/s]

Processing:   1%|          | 11/1134 [00:00<00:56, 19.92it/s]

Parsing error message empty range for randrange() (1, 1, 0)
Parsing error message empty range for randrange() (1, 1, 0)


Processing:   2%|▏         | 20/1134 [00:01<01:52,  9.93it/s]

Parsing error message empty range for randrange() (1, 1, 0)


...

Processing:  99%|█████████▉| 1121/1134 [03:58<00:02,  4.90it/s]

Parsing error message unrecognized expression


Processing: 100%|█████████▉| 1131/1134 [03:59<00:00,  5.16it/s]

Parsing error message 'NoneType' object has no attribute 'line'


Processing: 100%|██████████| 1134/1134 [03:59<00:00,  4.73it/s]


In [ ]:
concatenated_df.dropna(inplace=True)
concatenated_df.to_csv("final_non_vuln_code.csv", index=False)

In [22]:
import pandas as pd

# Generating dummy data so we can concat it
concatenated_df = pd.read_csv("final_non_vuln_code.csv")

dummy_data = [' ' for x in range(len(concatenated_df))]
concatenated_df['explanation'] = dummy_data

In [23]:
vuln_code_df

,file_name,vuln_title,severity,description,vuln_loc,vuln_explanation,vuln_recommendation,vuln_file_path,vuln_code
0,35.md,Unsafe cast in `ConcentratedLiquidityPool.burn...,high,The `ConcentratedLiquidityPool.burn` function ...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,The `ConcentratedLiquidityPool.burn` function ...,"Even though Solidity 0.8.x is used, type casts...",Web3Bugs/contracts/35/trident/contracts/pool/c...,function burn(bytes calldata data) public over...
1,35.md,Wrong usage of `positionId` in `ConcentratedLi...,high,#### Impact\nIn the `subscribe` function of `C...,{'vuln_file': ['ConcentratedLiquidityPoolManag...,In the `subscribe` function of `ConcentratedLi...,Change `positionId` to `incentiveId` in the re...,Web3Bugs/contracts/35/trident/contracts/pool/c...,"function subscribe(uint256 positionId, uint256..."
2,35.md,`ConcentratedLiquidityPoolManager`'s incentive...,high,The `ConcentratedLiquidityPoolManager` keeps a...,{'vuln_file': ['ConcentratedLiquidityPoolManag...,The `ConcentratedLiquidityPoolManager` keeps a...,"In `reclaimIncentive`, reduce `incentive.rewar...",Web3Bugs/contracts/35/trident/contracts/pool/c...,function reclaimIncentive(\n IConcentra...
3,35.md,Overflow in the `mint` function of `Concentrat...,high,#### Impact\nSimilar to a previous finding in ...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,Similar to a previous finding in the `IndexPoo...,Consider removing the `unchecked` statement to...,Web3Bugs/contracts/35/trident/contracts/pool/c...,function mint(bytes calldata data) public over...
4,35.md,Incorrect usage of typecasting in `_getAmounts...,high,#### Impact\nThe `_getAmountsForLiquidity` fun...,{'vuln_file': ['ConcentratedLiquidityPool.sol'...,The `_getAmountsForLiquidity` function of `Con...,Check whether the result of `DyDxMath.getDy` o...,Web3Bugs/contracts/35/trident/contracts/librar...,function _getAmountsForLiquidity(\n uin...
...,...,...,...,...,...,...,...,...,...
942,52.md,Lacking Validation Of Chainlink' Oracle Queries,medium,#### Impact\n\n`TwapOracle.consult()` is missi...,"{'vuln_file': ['TwapOracle.sol', 'GasThrottle....",TwapOracle.consult() is missing additional val...,Consider validating the output of latestRoundD...,Web3Bugs/contracts/52/contracts/twap/TwapOracl...,function consult(address token) public view re...
943,52.md,Vests can be denied,medium,The `LinearVesting.vestFor` function (which is...,"{'vuln_file': ['LinearVesting.sol'], 'vuln_fun...",The `LinearVesting.vestFor` function (which is...,There are several ways to mitigate this.\nThe ...,Web3Bugs/contracts/52/contracts/tokens/vesting...,"function vestFor(address user, uint256 amount)..."
944,52.md,Tokens with fee on transfer are not supported,medium,There are ERC20 tokens that charge fee for eve...,"{'vuln_file': ['BasePoolV2.sol'], 'vuln_functi...",There are ERC20 tokens that charge fee for eve...,Consider calling `balanceOf()` to get the actu...,Web3Bugs/contracts/52/contracts/dex-v2/pool/Ba...,"function mint(\n IERC20 foreignAsset,\n..."
945,52.md,VaderPoolV2.rescue results in loss of funds ra...,medium,#### Impact\n\nAny unaccounted for tokens on `...,"{'vuln_file': ['BasePoolV2.sol'], 'vuln_functi...",Any unaccounted for tokens on `VaderPoolV2` ca...,Permission this function to only allow Vader g...,Web3Bugs/contracts/52/contracts/dex-v2/pool/Ba...,function rescue(IERC20 foreignAsset) external ...


In [24]:
final_vuln = pd.DataFrame()

final_vuln['project_id'] = vuln_code_df['file_name'].apply(lambda x: x[:-3]).astype(int)
final_vuln['code'] = vuln_code_df['vuln_code']
final_vuln['code_analysis'] = vuln_code_df.apply(lambda x: f"{x['vuln_title']}\n{x['vuln_explanation']}", axis=1)
final_vuln['vulnerable'] = [1] * len(vuln_code_df)

final_non_vuln = pd.DataFrame()

final_non_vuln['project_id'] = concatenated_df['project_id'].astype(int)
final_non_vuln['code'] = concatenated_df['clean_code']
final_non_vuln['code_analysis'] = concatenated_df['explanation']
final_non_vuln['vulnerable'] = [0] * len(concatenated_df)

final_df = pd.concat([final_non_vuln, final_vuln], ignore_index=True)
final_df.sort_values("project_id", inplace=True)

In [25]:
final_df['code'] = final_df['code'].str.replace(r'// Error: Function body might be cropped\.', '', regex=True)

final_df.to_csv("final_dataset.csv", index=False)